# Пример решения

In [41]:
import pandas
import xarray
import requests
import datetime
import numpy as np
from tqdm import tqdm_notebook as tqdm

## NCEP Dataset

Погодные данные из проекта [NCEP Reanalysis 2](https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis2.html) — усреднённые за день температура воздуха, относительная влажность и компоненты ветра. Данные можно получить с 1979 года.

Загрузите наборы данных в каталог `data/ncep`:
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/air.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/uwnd.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/rhum.2018.nc

In [32]:
ncep_data = []
year = 2018
for var in ('air', 'uwnd', 'rhum'):
    dataset_filename = 'data/ncep/{}.{}.nc'.format(var, year)
    ncep_data.append(xarray.open_dataset(dataset_filename))
ncep_data = xarray.merge(ncep_data)

/usr/local/lib/python3.7/site-packages/xarray/conventions.py:401: SerializationWarning: variable 'air' has multiple fill values {-99, 32767}, decoding all values to NaN.
  stack_char_dim=stack_char_dim, use_cftime=use_cftime)
/usr/local/lib/python3.7/site-packages/xarray/conventions.py:401: SerializationWarning: variable 'uwnd' has multiple fill values {-99, 32767}, decoding all values to NaN.
  stack_char_dim=stack_char_dim, use_cftime=use_cftime)
/usr/local/lib/python3.7/site-packages/xarray/conventions.py:401: SerializationWarning: variable 'rhum' has multiple fill values {-99, 32767}, decoding all values to NaN.
  stack_char_dim=stack_char_dim, use_cftime=use_cftime)


## Набор признаков на основе данных NCEP

Ищем наиболее близкий к точке узел сетки в наборе NCEP, в качестве признаков значения переменных зарегистрированные в день регистрации точки и агрегированные показатели за период от 1 до 3х недель до момента регистрации точки.

In [37]:
def extract_features(row):
    point = ncep_data.sel(
        lon=row['longitude'],
        lat=row['latitude'],
        level=1000,
        method='nearest',
    )

    p1w = point.rolling(time=7).mean()
    p2w = point.rolling(time=14).mean()
    p3w = point.rolling(time=21).mean()
    
    date = row['date']
    v = point.sel(time=date)
    v1w = p1w.sel(time=date)
    v2w = p2w.sel(time=date)
    v3w = p3w.sel(time=date)
    
    return {
        'fire_type': row['fire_type'],
        'fire_type_name': row['fire_type_name'],
        'date': row['date'], 
        'temperature': v.air.values.item(0),
        'humidity': v.rhum.values.item(0),
        'uwind': v.uwnd.values.item(0),
        't1w': v1w.air.values.item(0),
        't2w': v2w.air.values.item(0),
        't3w': v3w.air.values.item(0),
        'h1w': v1w.rhum.values.item(0),
        'h2w': v2w.rhum.values.item(0),
        'h3w': v3w.rhum.values.item(0)
    }

## Выборка для обучения

In [38]:
df_train = pandas.read_csv('data/wildfires_train.csv')
df_subsample = df_train.query('(date > "2018") & (date < "2019")').sample(n=2000)

df_features = []
for i, row in tqdm(df_subsample.iterrows(), total=df_subsample.shape[0]):
    features = extract_features(row)
    df_features.append(features)
df_features = pandas.DataFrame(df_features)

## Обучение классификатора

In [42]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [53]:
X = df_features.iloc[:, 3:].fillna(0)
y = df_features['fire_type']

In [55]:
fire_classifier = GradientBoostingClassifier()

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [65]:
cross_val_score(
    fire_classifier, 
    X, y, 
    cv=5, 
)

array([0.44691358, 0.49875931, 0.4375    , 0.4559194 , 0.44810127])

In [67]:
fire_classifier.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

## Решение для отправки в систему

In [84]:
import pickle

with open('solution/model.pickle', 'wb') as fout:
    pickle.dump(fire_classifier, fout, protocol=pickle.HIGHEST_PROTOCOL)

## Прогнозирование на новых данных

In [73]:
df_predictions = pandas.DataFrame(
    fire_classifier.predict_proba(X),
    index=df_features.index,
    columns=[
        'fire_{}_prob'.format(class_id)
        for class_id in fire_classifier.classes_
    ],
)
df_predictions.head()

,fire_1_prob,fire_2_prob,fire_3_prob,fire_4_prob,fire_5_prob,fire_6_prob,fire_8_prob,fire_9_prob,fire_10_prob,fire_11_prob
0,0.001925,0.000094,0.061281,0.009380,0.000272,0.056167,0.028427,0.264281,0.543845,0.034329
1,0.000205,0.000022,0.003723,0.020655,0.000044,0.021186,0.003565,0.011526,0.926914,0.012161
2,0.002121,0.000045,0.025576,0.004506,0.000302,0.026830,0.006658,0.151995,0.036744,0.745223
3,0.001022,0.000102,0.028525,0.031514,0.000301,0.590062,0.017985,0.119299,0.129594,0.081595
4,0.001839,0.000350,0.014385,0.030361,0.002830,0.248328,0.016319,0.169094,0.467566,0.048927


In [77]:
df_predictions.to_csv('data/sample_predictions.csv', index_label='point_id')